In [1]:
# %%
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
import pickle


In [3]:

# Função para normalizar o texto
def normalizar_texto(texto):
    return texto.strip().lower()

# Carregar o dataset
dataset = pd.read_csv('problemas_combinados_2.csv', sep=',')

# Verificar as primeiras linhas do dataset
print("Primeiras linhas do dataset original:")
print(dataset.head())

# Unificar as colunas de quilometragem em uma única coluna
dataset_melted = pd.melt(
    dataset,
    id_vars=['Problema', 'Possíveis causas'],
    value_vars=['20.000km', '50.000km', '100.000km+'],
    var_name='Quilometragem',
    value_name='Probabilidade'
)



Primeiras linhas do dataset original:
                       Problema  \
0         Ar condicionado fraco   
1  Ar-condicionado não funciona   
2     Barulho estranho no motor   
3              Barulho no motor   
4    Carro puxando para um lado   

                                    Possíveis causas  20.000km  50.000km  \
0  Filtro sujo, Compressor com mal funcionamento,...       0.3       0.5   
1                              Filtro de ar entupido       0.2       0.4   
2  Correia dentada desgastada, Problemas no alter...       0.2       0.5   
3  Pressão incorreta, Correia dentada desgastada,...       0.2       0.4   
4        Desalinhamento, Pressão dos pneus desiguais       0.3       0.5   

   100.000km+  
0         0.7  
1         0.6  
2         0.8  
3         0.7  
4         0.6  


# Unificar as colunas de quilometragem em uma única coluna
O método pd.melt é utilizado para transformar o formato do dataframe, criando uma nova coluna 'Quilometragem' que contém as diferentes quilometragens e outra coluna 'Probabilidade' que contém os valores correspondentes.

Isso é útil para análises posteriores, onde podemos querer trabalhar com os dados de quilometragem de forma mais uniforme.

In [4]:
# Manter apenas as linhas com 'Probabilidade' > 0
dataset_melted = dataset_melted[dataset_melted['Probabilidade'] > 0]

# Normalizar os textos
dataset_melted['Problema'] = dataset_melted['Problema'].apply(normalizar_texto)
dataset_melted['Quilometragem'] = dataset_melted['Quilometragem'].apply(normalizar_texto)
dataset_melted['Possíveis causas'] = dataset_melted['Possíveis causas'].apply(
    lambda x: [normalizar_texto(causa) for causa in x.split(', ')]
)

# Combinar 'Problema' e 'Quilometragem' em uma única coluna de texto
dataset_melted['Entrada'] = dataset_melted['Problema'] + ' ' + dataset_melted['Quilometragem']

# Vetorizar a entrada usando TF-IDF com n-grams e ajustes
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=1,
    max_df=0.8,
    strip_accents='unicode',
    lowercase=True
)
X = vectorizer.fit_transform(dataset_melted['Entrada'])

# Binarizar 'Possíveis causas' para classificação multilabel
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(dataset_melted['Possíveis causas'])

# Verificar a distribuição das classes
print("\nDistribuição das classes após binarização:")
print(pd.DataFrame(y, columns=mlb.classes_).sum())

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)




Distribuição das classes após binarização:
acúmulo de sujeira                           3
alinhamento da porta incorreto               3
alinhamento incorreto                        9
alternador com defeito                       3
alternador com problema                      3
                                            ..
velas de ignição desgastadas                 3
ventoinha do radiador com problema           3
válvula de controle de ar inativa            3
válvula de controle de calor com problema    3
óleo baixo                                   3
Length: 116, dtype: int64


O código apresentado realiza um pré-processamento de dados para um modelo de aprendizado de máquina, começando pela filtragem de linhas com probabilidade maior que zero. Essa etapa é crucial para garantir que apenas dados relevantes sejam analisados, melhorando a qualidade do modelo. Em seguida, o código normaliza os textos das colunas "Problema" e "Quilometragem", removendo espaços em branco e convertendo tudo para minúsculas, o que ajuda a padronizar os dados. As "Possíveis causas" são também normalizadas, sendo divididas em listas para facilitar o manuseio. Posteriormente, as colunas "Problema" e "Quilometragem" são combinadas em uma nova coluna chamada "Entrada", que será utilizada para a vetorização.

A vetorização é feita utilizando o TfidfVectorizer, que transforma o texto em uma matriz de características, levando em consideração unigramas e bigramas. Essa abordagem permite capturar tanto palavras isoladas quanto pares de palavras, o que pode melhorar a eficácia do modelo. Após a vetorização, as "Possíveis causas" são binarizadas usando o MultiLabelBinarizer, criando uma matriz onde cada coluna representa uma classe e os valores indicam a presença ou ausência das causas. Finalmente, os dados são divididos em conjuntos de treinamento e teste, permitindo a avaliação do desempenho do modelo em dados não vistos. Essa preparação metódica é fundamental para o sucesso de qualquer projeto de aprendizado de máquina.

In [5]:
# Inicializar o modelo com OneVsRestClassifier e LogisticRegression com class_weight='balanced'
model = OneVsRestClassifier(
    LogisticRegression(max_iter=2000, solver='liblinear', class_weight='balanced')
)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

# Salvar o modelo, o vetorizador e o binarizador em um arquivo pickle
with open('modelo_causas_aprimorado.pkl', 'wb') as f:
    pickle.dump({
        'model': model,
        'vectorizer': vectorizer,
        'mlb': mlb
    }, f)

print("Modelo aprimorado salvo como 'modelo_causas_aprimorado.pkl'")


Relatório de Classificação:
                                           precision    recall  f1-score   support

                       acúmulo de sujeira       0.00      0.00      0.00         0
           alinhamento da porta incorreto       0.00      0.00      0.00         0
                    alinhamento incorreto       1.00      1.00      1.00         5
                   alternador com defeito       1.00      1.00      1.00         2
                  alternador com problema       1.00      1.00      1.00         1
                   amortecedor danificado       1.00      1.00      1.00         1
                amortecedores desgastados       1.00      1.00      1.00         3
                 antena de gps danificada       0.00      0.00      0.00         0
                   bateria da chave fraca       0.00      0.00      0.00         0
                     bateria descarregada       0.00      0.00      0.00         0
                            bateria fraca       1.00     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


O código apresentado realiza a inicialização e o treinamento de um modelo de aprendizado de máquina para classificação multilabel utilizando o `OneVsRestClassifier` com uma regressão logística. O `class_weight='balanced'` é utilizado para lidar com classes desbalanceadas, ajustando automaticamente os pesos das classes com base em suas frequências no conjunto de dados. Após a inicialização, o modelo é treinado com os dados de treinamento (`X_train` e `y_train`). Esse passo é fundamental, pois permite que o modelo aprenda a identificar padrões nos dados que estão associados a cada uma das possíveis causas.

Uma vez que o modelo está treinado, ele é utilizado para fazer previsões no conjunto de teste (`X_test`). A eficácia do modelo é avaliada através do `classification_report`, que fornece métricas detalhadas de desempenho, como precisão, recall e F1-score para cada classe. Isso ajuda a entender como o modelo se comporta em relação a diferentes causas. Por fim, o modelo, o vetorizador e o binarizador são salvos em um arquivo pickle. Isso permite que o modelo seja reutilizado sem a necessidade de re-treinamento, facilitando a implementação em aplicações futuras. A mensagem de confirmação indica que o processo foi concluído com sucesso, sinalizando que o modelo aprimorado está agora pronto para uso.